# Basic plotting tools

hv resources
- Gallery: https://holoviews.org/reference/
- https://holoviews.org/user_guide/Customizing_Plots.html
- https://holoviews.org/user_guide/Style_Mapping.html
- https://holoviews.org/user_guide/Colormaps.html
- https://holoviews.org/user_guide/Plotting_with_Bokeh.html

In [ ]:
import holoviews as hv

import hv_anndata
from hv_anndata import ACCESSOR as A
from hv_anndata import register

register()

hv.extension("bokeh")

In [ ]:
import numpy as np
import scanpy as sc

rng = np.random.default_rng()

In [ ]:
adata = sc.datasets.pbmc68k_reduced()
adata.layers["counts"] = adata.raw.X
del adata.raw
sc.tl.umap(adata)
adata

{func}`scanpy.pl.scatter`

missing features:
- `use_raw` (deprecated!)
- `na_color` (not super important)

missing convenience:
- `basis` for easy X&Y

In [ ]:
# add NAs to check how missing values look
adata_scatter = adata.copy()
adata_scatter.obs.loc[
    (
        (adata_scatter.obs["bulk_labels"] == "Dendritic")
        & rng.choice([True, False], size=len(adata_scatter))
    ),
    "bulk_labels",
] = np.nan

hv.Scatter(
    adata_scatter, A.obsm["X_umap"][0], [A.obsm["X_umap"][1], A.obs["bulk_labels"]]
).opts(
    color=A.obs["bulk_labels"], cmap="tab10", aspect="square", legend_position="right"
)

{func}`scanpy.pl.heatmap`

missing:
- `groupby` / [TickBar](https://github.com/holoviz/holoviews/issues/6658)
- `standard_scale` (see implemantation in {class}`hv_anndata.Dotmap`)
  - maybe done using [`dim` expressions](https://holoviews.org/user_guide/Transforming_Elements.html)?
- dendrogram doesn’t work on heatmap (again ndim problem: dendrogram should tread `(n, 1)` as 1D)

In [ ]:
markers = ["C1QA", "PSAP", "CD79A", "CD79B", "CST3", "LYZ"]
hm = hv.HeatMap(
    adata[:, markers],
    [A.obs.index, A.var.index],
    [A[:, :], A.obs["bulk_labels"], A.obs["n_counts"]],
).opts(xticks=0, colorbar=True, frame_width=800, frame_height=100)
try:
    hm = hv.operation.dendrogram(
        hm,
        adjoint_dims=[A.obs["bulk_labels"]],
        main_dim=A.obs["n_counts"],
        linkage_metric="euclidean",
    )
except Exception:  # noqa: BLE001
    import traceback

    traceback.print_exc()
hm

{func}`scanpy.pl.dotplot`

missing:
- `var_group_*` (highlight groups of `var_names` by drawing brackets)

In [ ]:
markers = ["C1QA", "PSAP", "CD79A", "CD79B", "CST3", "LYZ"]
dm = hv_anndata.Dotmap(adata=adata, marker_genes=markers, groupby="bulk_labels")
dm

In [ ]:
hv.operation.dendrogram(
    dm.plot(), adjoint_dims=["cluster"], main_dim="mean_expression", invert=True
)

{func}`scanpy.pl.tracksplot`

missing:
- `shared_xaxis=False` doesn’t seem to work on GridSpace
- GridSpace can only be 2D or row-only, not col-only
- [GridSpace `show_legend` broken](https://github.com/holoviz/holoviews/issues/5438)

In [ ]:
markers = ["C1QA", "PSAP", "CD79A", "CD79B", "CST3", "LYZ"]
hv.NdLayout(
    {
        m: hv.Curve(adata, [A.obs.index], [A[:, m], A.obs["bulk_labels"]])
        .opts(
            xticks=0,
            xlabel="",
            ylabel=m,
            title="",
            frame_height=50,
            frame_width=900,
            show_legend=False,
        )
        .groupby(A.obs["bulk_labels"], hv.NdOverlay)
        for m in markers
    },
    kdims=["marker"],
).cols(1)

In [ ]:
markers = ["C1QA", "PSAP", "CD79A", "CD79B", "CST3", "LYZ"]
hv.GridSpace(
    {
        (0, m): hv.Curve(adata, [A.obs.index], [A[:, m], A.obs["bulk_labels"]])
        .opts(frame_height=50, frame_width=900)
        .groupby(A.obs["bulk_labels"], hv.NdOverlay)
        for m in markers
    },
    kdims=["_", "marker"],
).opts(show_legend=True, xaxis=None)

{func}`scanpy.pl.violin`

missing:
- `density_norm`

In [ ]:
hv.Layout([
    hv.Violin(adata, vdims=[A.obs[col]]).opts(
        ylabel="", title=col, show_grid=True, ylim=(0, None)
    )
    for col in ["percent_mito", "n_counts", "n_genes"]
]).opts(axiswise=True)

In [ ]:
# hv.Violin(adata, [A.obs["bulk_labels"]], [A.obs["S_score"]]).groupby(
#    A.obs["bulk_labels"], hv.NdOverlay
# ).opts(width=500, xrotation=30)
hv.Violin(adata, [A.obs["bulk_labels"]], [A.obs["S_score"]]).opts(
    width=500, xrotation=30, violin_fill_color=A.obs["bulk_labels"]
)

{func}`scanpy.pl.stacked_violin`

missing:
- see tracksplot above
- can’t do `.hist()` or `.operations.dendrogram` on GridSpace
- slow!

In [ ]:
markers = ["C1QA", "PSAP", "CD79A", "CD79B", "CST3", "LYZ"]
hv.GridSpace(
    {
        (marker, label): hv.Violin(
            adata[adata.obs["bulk_labels"] == label].copy(), vdims=[A[:, marker]]
        ).opts(frame_height=50, frame_width=50, inner=None)
        for marker in markers
        for label in adata.obs["bulk_labels"].cat.categories
    },
    ["marker", "bulk label"],
)

{func}`scanpy.pl.matrixplot`

missing:
- `aspect="equal"` breaks (bokeh tries to divide `None/None`)
- hist doesn’t align properly

In [ ]:
markers = ["C1QA", "PSAP", "CD79A", "CD79B", "CST3", "LYZ"]
hv.HeatMap(
    sc.get.aggregate(adata[:, markers], "bulk_labels", "mean"),
    [A.obs.index, A.var.index],
    [A.layers["mean"][:, :]],
).opts(xrotation=30, frame_height=180, frame_width=300).hist(num_bins=6)

{func}`scanpy.pl.clustermap`

missing:
- TickBar (see above)

In [ ]:
hv_anndata.ClusterMap(adata=adata)

{func}`scanpy.pl.dendrogram`: {class}`holoviews.operation.dendrogram`